# Imports

In [3]:
import json
import pandas as pd
import matplotlib as mpl
from matplotlib import pyplot as plt
from sklearn import preprocessing as preproc
from sklearn.decomposition import PCA
import numpy as np

# import tqdm to notebook
from tqdm.notebook import tqdm

# https://stackoverflow.com/questions/52285104/3d-scatterplots-with-hue-colormap-and-legend
from mpl_toolkits.mplot3d import Axes3D

import seaborn as sns

try:
  from google.colab import drive
  drive.mount('/content/drive')

  COLAB = True
except ModuleNotFoundError:
  COLAB = False

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from minisom import MiniSom

import tensorflow as tf
from tensorflow import keras


import matplotlib.pyplot as plt
from matplotlib.patches import RegularPolygon, Ellipse
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib import cm, colorbar
from matplotlib.lines import Line2D

# forcing tensorflow to use CPU

from datetime import datetime

%matplotlib inline

2024-03-19 15:51:16.277451: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-19 15:51:16.302380: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-19 15:51:16.709388: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Load Data

In [4]:
df = pd.read_parquet("diabetic_data_cleaned_pca_som.parquet")

# fix some types that parquet didn't get right
df["admission_type_id"] = df["admission_type_id"].astype("category")
df["discharge_disposition_id"] = df["discharge_disposition_id"].astype("category")
df["admission_source_id"] = df["admission_source_id"].astype("category")

display(df.shape)
display(df.dtypes)
display(df.head(2).T)

(100244, 40)

race                          category
gender                        category
age                           category
admission_type_id             category
discharge_disposition_id      category
admission_source_id           category
time_in_hospital                 int64
payer_code                    category
medical_specialty             category
num_lab_procedures               int64
num_procedures                   int64
num_medications                  int64
number_outpatient                int64
number_emergency                 int64
number_inpatient                 int64
diag_1                        category
diag_2                        category
diag_3                        category
number_diagnoses                 int64
metformin                     category
repaglinide                   category
glimepiride                   category
glipizide                     category
glyburide                     category
pioglitazone                  category
rosiglitazone            

,0,1
race,Caucasian,AfricanAmerican
gender,Female,Female
age,[10-20),[20-30)
admission_type_id,1,1
discharge_disposition_id,1,1
admission_source_id,7,7
time_in_hospital,3,2
payer_code,NaN,NaN
medical_specialty,NaN,NaN
num_lab_procedures,59,11


In [5]:
with open("data_dictionary.json", "r") as f:
  data_info = json.load(f)

# NUM_COLUMNS = data_info["Numeric Columns"]
NOM_COLUMNS = data_info["Nominal Columns"]
ORD_COLUMNS = data_info["Ordinal Columns"]
TGT = data_info["Target"]



# Auto Encoders

In [6]:
df_temp = df.copy()

CAT_COLUMNS = NOM_COLUMNS + ORD_COLUMNS
CAT_COLUMNS.remove("payer_code")
CAT_COLUMNS.remove("medical_specialty")

df_temp = df_temp[CAT_COLUMNS]
for col in CAT_COLUMNS:
  df_temp[col] = df_temp[col].astype("category")

ohe = OneHotEncoder(drop=None, sparse_output=False)
temp = ohe.fit_transform(df_temp)

temp.shape

(100244, 2355)

In [10]:
def train_autoenc(train_data, df_ref, latent_vars, learning_rate, epochs):
    input_shape = (2355,)

    inp = keras.Input(shape=input_shape)

    enc = keras.layers.Dense(400, activation="relu")(inp)
    enc = keras.layers.Dense(150, activation="relu")(enc)
    enc = keras.layers.Dense(latent_vars, activation="relu", name="encoder_out")(enc)

    dec = keras.layers.Dense(latent_vars, activation="relu")(enc)
    dec = keras.layers.Dense(150, activation="relu")(dec)
    dec = keras.layers.Dense(400, activation="relu")(dec)

    outputs = []
    outs = 0

    # create output layers
    for c in df_ref:
        out_num = df_ref[c].nunique()
        if df_ref[c].isna().sum() > 0:
            out_num += 1

        out_temp = keras.layers.Dense(out_num, activation="softmax", name=f"out_{c}")(dec)
        outs += out_num
        outputs.append(out_temp)

    auto_encoder = keras.Model(inp, outputs)
    auto_encoder.summary()
    
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        learning_rate,
        decay_steps=1600,  # each, more or less, 3 epochs (batch size 32)
        decay_rate=0.98,  # reduce to 10% in each +- 120 epochs
        staircase=True)  # If True, learning rate decays in a discrete staircase, else smoothly

    opt = keras.optimizers.Adam(learning_rate=lr_schedule)

    # opt = keras.optimizers.Adam(learning_rate=learning_rate)

    losses = {f"out_{c}": "categorical_crossentropy" for c in df_ref}
    metrics = {f"out_{c}": ["accuracy"] for c in df_ref}

    auto_encoder.compile(optimizer=opt, loss=losses, metrics=metrics)
    run_id = datetime.now().strftime("%Y%m%d-%H%M%S")
    log_dir = "logs/fit/" + run_id

    print(log_dir)
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

    targets = []

    idx = 0
    for o in outputs:
        tgt = train_data[:, idx:idx+o.shape[1]]
        targets.append(tgt)
        idx += o.shape[1]
        
    hist = auto_encoder.fit(train_data, targets,
                            epochs=epochs, batch_size=64, shuffle=True,
                            callbacks=[tensorboard_callback],
                            verbose=2)

    with open(f"auto_encoder_{run_id}.json", "w") as f:
        d = hist.history
        d["run_id"] = run_id
        d["latent_variables"] = latent_vars
        d["learning_rate"] = learning_rate    
        d["epochs"] = epochs    
        json.dump(d, f, indent=4)

        # save weights
        auto_encoder.save(f"auto_encoder_{run_id}.h5")
    
    return auto_encoder


In [11]:
ae = train_autoenc(temp, df_temp, latent_vars=12, learning_rate=0.001, epochs=150)

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 2355)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 400)       │    942,400 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_8 (Dense)     │ (None, 150)       │     60,150 │ dense_7[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder_out (Dense) │ (None, 12)        │      1,812 │ dense_8[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, 12)        │        156 │ encoder_out[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_10 (Dense)    │ (None, 150)       │      1,950 │ dense_9[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_11 (Dense)    │ (None, 400)       │     60,400 │ dense_10[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ out_race (Dense)    │ (None, 5)         │      2,005 │ dense_11[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ out_gender (Dense)  │ (None, 2)         │        802 │ dense_11[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ out_diag_1 (Dense)  │ (None, 715)       │    286,715 │ dense_11[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ out_diag_2 (Dense)  │ (None, 743)       │    297,943 │ dense_11[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ out_diag_3 (Dense)  │ (None, 789)       │    316,389 │ dense_11[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ out_change (Dense)  │ (None, 2)         │        802 │ dense_11[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ out_diabetesMed     │ (None, 2)         │        802 │ dense_11[0][0]    │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ out_admission_type… │ (None, 8)         │      3,208 │ dense_11[0][0]    │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ out_discharge_disp… │ (None, 26)        │     10,426 │ dense_11[0][0]    │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ out_admission_sour… │ (None, 17)        │      6,817 │ dense_11[0][0]    │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ out_metformin       │ (None, 4)         │      1,604 │ dense_11[0][0]    │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ out_repaglinide     │ (None, 4)         │      1,604 │ dense_11[0][0]    │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ out_glimepiride     │ (None, 4)         │      1,604 │ dense_11[0][0]    │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ out_glipizide       │ (None, 4)         │      1,604 │ dense_11[0][0]  

 Total params: 2,011,223 (7.67 MB)

 Trainable params: 2,011,223 (7.67 MB)

 Non-trainable params: 0 (0.00 B)

logs/fit/20240319-155326
Epoch 1/150


I0000 00:00:1710863611.124386  974279 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_5859', 20 bytes spill stores, 20 bytes spill loads

I0000 00:00:1710863611.137724  974286 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_5859', 20 bytes spill stores, 20 bytes spill loads

I0000 00:00:1710863611.179228  974285 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_5859', 20 bytes spill stores, 20 bytes spill loads

I0000 00:00:1710863611.208803  974308 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_1474', 28 bytes spill stores, 28 bytes spill loads

I0000 00:00:1710863611.243705  974287 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_5859', 16 bytes spill stores, 16 bytes spill loads

I0000 00:00:1710863611.332833 

1567/1567 - 28s - 18ms/step - loss: 16.3855 - out_admission_source_id_accuracy: 0.8498 - out_admission_type_id_accuracy: 0.8374 - out_age_accuracy: 0.6859 - out_change_accuracy: 0.9733 - out_diabetesMed_accuracy: 0.9838 - out_diag_1_accuracy: 0.2022 - out_diag_2_accuracy: 0.1927 - out_diag_3_accuracy: 0.1866 - out_discharge_disposition_id_accuracy: 0.7928 - out_gender_accuracy: 0.9347 - out_glimepiride_accuracy: 0.9503 - out_glipizide_accuracy: 0.9041 - out_glyburide-metformin_accuracy: 0.9909 - out_glyburide_accuracy: 0.9224 - out_insulin_accuracy: 0.9017 - out_metformin_accuracy: 0.9367 - out_pioglitazone_accuracy: 0.9315 - out_race_accuracy: 0.9183 - out_repaglinide_accuracy: 0.9845 - out_rosiglitazone_accuracy: 0.9365
Epoch 2/150
1567/1567 - 5s - 3ms/step - loss: 12.0083 - out_admission_source_id_accuracy: 0.8905 - out_admission_type_id_accuracy: 0.8924 - out_age_accuracy: 0.9232 - out_change_accuracy: 0.9942 - out_diabetesMed_accuracy: 0.9980 - out_diag_1_accuracy: 0.3329 - out_di

In [12]:
encoder = keras.Model(inputs=ae.input, outputs=ae.get_layer("encoder_out").output)

In [27]:
ae_vars = encoder.predict(temp)
df_ae = pd.DataFrame(ae_vars, columns=[f"ae_temp_{i}" for i in range(ae_vars.shape[1])])

final_vars = []
for c in df_ae:
    if df_ae[c].max() - df_ae[c].min() == 0:
        continue

    final_vars.append(c)

df_ae = df_ae[final_vars].rename(columns={c: f"ae_{i}" for i, c in enumerate(final_vars)})

3133/3133 ━━━━━━━━━━━━━━━━━━━━ 1s 359us/step


In [30]:
df_final = df.join(df_ae)
display(df_final.describe().T)

filename = "diabetic_data_cleaned_pca_som_ae"

df.to_csv(f"{filename}.csv", index=False)
df.to_parquet(f"{filename}.parquet")

,count,mean,std,min,25%,50%,75%,max
time_in_hospital,100244.0,4.420275e+00,2.991273,1.000000,2.000000,4.000000,6.000000,14.000000
num_lab_procedures,100244.0,4.317372e+01,19.691484,1.000000,32.000000,44.000000,57.000000,132.000000
num_procedures,100244.0,1.350275e+00,1.710913,0.000000,0.000000,1.000000,2.000000,6.000000
num_medications,100244.0,1.611782e+01,8.116424,1.000000,11.000000,15.000000,20.000000,81.000000
number_outpatient,100244.0,3.725610e-01,1.274074,0.000000,0.000000,0.000000,0.000000,42.000000
number_emergency,100244.0,1.990144e-01,0.933541,0.000000,0.000000,0.000000,0.000000,76.000000
number_inpatient,100244.0,6.394697e-01,1.262773,0.000000,0.000000,0.000000,1.000000,21.000000
number_diagnoses,100244.0,7.497007e+00,1.839537,3.000000,6.000000,8.000000,9.000000,16.000000
PC1,100244.0,-5.091051e-17,0.299783,-0.400778,-0.232916,-0.088537,0.155953,1.159058
PC2,100244.0,-6.268567e-17,0.230848,-0.741977,-0.142057,-0.017638,0.132432,0.844482


## Conclusion

This notebook finishes adding 11 new columns. The auto-encoder model could replace the 22 original categorical variables (with more than 2400 combined levels) with 11 numerical variables. It is an unsupervised learning technique since the goal is to recreate the original data (without the target!) with only these 11 numerical variables. It achieved an excellent result. This set of 11 variables can be used in place of the original categorical variables if you want (the same for SOM_JAC and SOM_EUC).